<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Adapt_feature_based_PRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PRED**

In [ ]:
!pip install adapt

In [3]:
from sklearn.linear_model import RidgeClassifier
from adapt.utils import make_classification_da
from adapt.feature_based import PRED
Xs, ys, Xt, yt = make_classification_da()
model = PRED(RidgeClassifier(0.), Xt=Xt[[1, -1, -2]], yt=yt[[1, -1, -2]],
             pretrain=True, verbose=0, random_state=0)
model.fit(Xs, ys)
model.score(Xt, yt)

0.77

On reproduit le code sur un nouveau jeu de donnée de notre choix

In [4]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_breast_cancer
from adapt.feature_based import PRED
from sklearn.model_selection import train_test_split

# Charger le jeu de données sur le cancer du sein
data = load_breast_cancer()
X, y = data.data, data.target

# Séparer les données en ensemble source (Xs, ys) et ensemble cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle PRED avec RidgeClassifier
model = PRED(RidgeClassifier(alpha=1.0), Xt=Xt[:3], yt=yt[:3],
             pretrain=True, verbose=0, random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Score: 0.9333333333333333


Ce score est encourageant et montre que le modèle est performant dans cette tâche spécifique, mais une analyse plus approfondie avec des métriques complémentaires et des tests sur d'autres données permettrait de confirmer la validité et la robustesse de cette performance.

# **FA**

In [9]:
from sklearn.linear_model import RidgeClassifier
from adapt.utils import make_classification_da
from adapt.feature_based import FA
Xs, ys, Xt, yt = make_classification_da()
model = FA(RidgeClassifier(), Xt=Xt[:10], yt=yt[:10], random_state=0)
model.fit(Xs, ys)
model.score(Xt, yt)

Fit transform...
Previous shape: (100, 2)
New shape: (110, 6)
Fit Estimator...


0.92

In [11]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_digits
from adapt.feature_based import FA
from sklearn.model_selection import train_test_split

# Charger le jeu de données digits
data = load_digits()
X, y = data.data, data.target

# Diviser les données en ensembles source (Xs, ys) et cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle FA avec RidgeClassifier
# En utilisant un sous-ensemble de Xt et yt pour l'adaptation de domaine
model = FA(RidgeClassifier(), Xt=Xt[:10], yt=yt[:10], random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Fit transform...
Previous shape: (898, 64)
New shape: (908, 192)
Fit Estimator...
Score: 0.6162402669632926


# **CORAL**

In [10]:
from sklearn.linear_model import RidgeClassifier
from adapt.utils import make_classification_da
from adapt.feature_based import CORAL
Xs, ys, Xt, yt = make_classification_da()
model = CORAL(RidgeClassifier(), Xt=Xt, random_state=0)
model.fit(Xs, ys)
model.score(Xt, yt)

Fit transform...
Previous covariance difference: 0.013181
New covariance difference: 0.000004
Fit Estimator...


0.86

In [12]:
from sklearn.linear_model import RidgeClassifier
from sklearn.datasets import load_wine
from adapt.feature_based import CORAL
from sklearn.model_selection import train_test_split

# Charger le jeu de données wine
data = load_wine()
X, y = data.data, data.target

# Diviser les données en ensembles source (Xs, ys) et cible (Xt, yt)
Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.5, random_state=0)

# Initialiser et entraîner le modèle CORAL avec RidgeClassifier
model = CORAL(RidgeClassifier(), Xt=Xt, random_state=0)
model.fit(Xs, ys)

# Évaluer le modèle
score = model.score(Xt, yt)
print("Score:", score)


Fit transform...
Previous covariance difference: 35.215205
New covariance difference: 0.000002
Fit Estimator...
Score: 0.8426966292134831
